In [1]:
from lyrics_to_melody import Lyrics_match, get_chorus, lm_score, fill_template, splice, polish_chord, chord_truc, polish, not_mono, not_duplicate, save_demo
import random
import sqlite3
import re
from fairseq.models.transformer_lm import TransformerLanguageModel
import numpy as np
from math import ceil
import copy
from midiutil.MidiFile import MIDIFile
import miditoolkit

ModuleNotFoundError: No module named 'torch._C'

# importing a database

In [2]:
# Загрузка базы мелодий
db_path = 'database/ROC.db'
conn = sqlite3.connect(db_path)
global c
c = conn.cursor()
print("Database connected")

Database connected


### select_melody function code

In [3]:
def select_melody(is_maj, is_chorus, length, last_bar, chord, chord_ptr, is_last_sentence):
    cursor = c.execute(
        "SELECT DISTINCT NOTES, CHORDS from MELOLIB  where LENGTH = '{}' and CHORUS = '{}' and MAJOR = '{}' ".format(
            length, is_chorus, is_maj))  # and MAJOR = '{}'
    candidates_bars = []
    print("Retrive melody...")
    for row in cursor:
        notes = row[0]
        cd_ = row[1]
        candidates_bars.append((notes, cd_))

    # Filter by chords.
    chord_list_ = chord.strip().split(' ')
    chord_list_ = chord_list_[chord_ptr:] + chord_list_[:chord_ptr]
    re_str = ''

    if not is_last_sentence:
        key = ''
    else:
        if is_maj:
            key = ' C:'
        else:
            key = ' A:m'

    # For the given chord progression, we generate a regex like:
    # A:m F: G: C: -> ^A:m( A:m)*( F:)+( G:)+( C:)*$|^A:m( A:m)*( F:)+( G:)*$|^A:m( A:m)*( F:)*$|^A:m( A:m)*$
    # Given the regex, we find matched pieces.
    # We design the regex like this because alternations in regular expressions are evaluated from left to right,
    # the piece with the most various chords will be selected, if there's any.
    for j in range(len(chord_list_), 0, -1):
        re_str += '^({}( {})*'.format(chord_list_[0], chord_list_[0])
        for idx in range(1, j):
            re_str += '( {})+'.format(chord_list_[idx])
        re_str = re_str[:-1]
        re_str += '*{})$|'.format(key)
    re_str = re_str[:-1]

    tmp_candidates = []
    for row in candidates_bars:
        if re.match(r'{}'.format(re_str), row[1]):
            tmp_candidates.append(row)

    if len(tmp_candidates) == 0:
        re_str = '^{}( {})*$'.format(chord_list_[-1], chord_list_[-1])
        for row in candidates_bars:
            if re.match(r'{}'.format(re_str), row[1]):
                tmp_candidates.append(row)

    if len(tmp_candidates) > 0:
        candidates_bars = tmp_candidates
    else:
        if is_maj:
            re_str = '^C:( C:)*$'
        else:
            re_str = '^A:m( A:m)*$'
        for row in candidates_bars:
            if re.match(r'{}'.format(re_str), row[1]):
                tmp_candidates.append(row)
        if len(tmp_candidates) > 0:
            candidates_bars = tmp_candidates

    candidates_cnt = len(candidates_bars)
    if candidates_cnt == 0:

        print('No Matched Rhythm as {}'.format(length))
        return []

    if last_bar == None:  # we are at the begining of a song, random select bars.

        print('Start a song...')

        def not_too_high(bar):
            notes = bar.split(' ')[:-1][3::5]
            notes = [int(x[6:]) for x in notes]
            for i in notes:
                if 57 > i or i > 66:
                    return False
            return True

        tmp = []
        for bar in candidates_bars:
            if not_too_high(bar[0]):
                tmp.append(bar)
        return tmp
    else:
        last_note = int(last_bar.split(' ')[-3][6:])
        # tendency
        selected_bars = []
        prefer_note = None

        # Major C
        if is_maj:
            if last_note % 12 == 2 or last_note % 12 == 9:
                prefer_note = last_note - 2
            elif last_note % 12 == 5:
                prefer_note = last_note - 1
            elif last_note % 12 == 11:
                prefer_note = last_note + 1
                # Minor A
        else:
            if last_note % 12 == 11 or last_note % 12 == 2:  # 2 -> 1, 4 -> 3
                prefer_note = last_note - 2
            elif last_note % 12 == 6:  # 6 -> 5
                prefer_note = last_note - 1
            elif last_note % 12 == 7:  # 7-> 1
                prefer_note = last_note + 2

        if prefer_note is not None:
            for x in candidates_bars:
                if x[0][0] == prefer_note:
                    selected_bars.append(x)
        if len(selected_bars) > 0:
            print('Filter by tendency...')
            candidates_bars = selected_bars

        selected_bars = []
        for bar in candidates_bars:
            first_pitch = int(bar[0].split(' ')[3][6:])
            if (first_pitch > last_note - 8 and first_pitch < last_note + 8):
                selected_bars.append(bar)
        if len(selected_bars) > 0:
            print('Filter by pitch range...')
            return selected_bars

    # No candidates yet? randomly return some.
    print("Randomly selected...")
    return candidates_bars

# Открываем файлы текста и аккордов

In [4]:
lyrics_path = '/home/marykorol/ROC/lyrics.txt'
with open(lyrics_path) as lyrics_file:
    lyrics = lyrics_file.readlines()[1]

lyrics

'Кто зна ет, за чем за жи га ют звёз ды? [sep] Для ко го о кон мос ковс ких свет и ко гда за кон чит ся зи ма? [sep] Кто зна ет, ка кие на вы ход ны е прог но зы? [sep] Где ку пить на э том ра йоне ночь ю? [sep] И се год ня как сыг рал Спар так? [sep] Всё рав но, кто ста нет пре зи ден том [sep] Всё рав но, ка ко го раз мера грудь [sep] Всё рав но, в чём си ла, за кем прав да [sep] Всё рав но, по ре зать и ли по лос нуть [sep] Всё рав но, сколь ко се год ня сто ит рубль [sep] Всё рав но, сколь ко ос та лось си га рет [sep] Всё рав но, ста рый ле вый и ли но вый пра вый [sep] Всё рав но, аб со лют но точ но, есть бог и ли нет [sep] И пусть со сут ин тел лек ту а лы [sep] Де пу та ты и ар тис ты [sep] И дёт на хуй Ве ра По лоз ко ва [sep] Му зы кан ты и дут и, ко неч но, жур на лис ты [sep] Всё рав но, тот на род и ли не тот [sep] Я не спе ци а лист, но вро де бы ва ют и по ху же [sep] Мне всё рав но, что по эт тут боль ше чем по эт [sep] Для ме ня есть ты и я, боль ше сов сем ник то не 

In [5]:
chord_path = '/home/marykorol/ROC/chord.txt'
with open(chord_path) as chord_file:
    chord = chord_file.readlines()[1]
    chord = chord[2:].strip()

print('CHORD:', chord)

CHORD: C: G: A:m F: C: G: A:m F: C: G: A:m F: C: G: A:m F: C: G: A:m F: C: G: A:m F: C: G: A:m F: C: G: A:m F: C: G: A:m F: C: G: A:m F: C: G: A:m F: C: G: A:m F: C: G: A:m F: C: G: A:m F: C: G: A:m F: C: G: A:m F: C: G: A:m F: C: G: A:m F: C: G: A:m F: C: G: A:m F: C: G: A:m F:


In [6]:
chord_ptr = 0
is_maj = 1 #(мажорный или минорный лад)

In [9]:
sentence = lyrics.strip().split('[sep]')
name = sentence[0]
sentence = [len(x.strip().split(' ')) * '_' for x in sentence] # parsing the syllables

In [11]:
len(sentence)

26

# Structure recognition

In [172]:
# structure recognition
parent, chorus_start, chorus_length = Lyrics_match(sentence)  # The last element must be -1, because the chord should go back to tonic

print('Struct Array: ', parent)

for test: [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 11, 12, 13, 14, -1, -1, -1, 18, 19, 20, 21, -1, -1, -1, -1, -1, -1, -1, -1, -1, 6, 7, 8, -1, 10, 11, 12, 13, 14, 11, 12, 13, 18, 19, 20, 21, 18, 19, 20, 21]
length: 55
Struct Array:  [-1, -2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -2, -1, -1, 11, 12, 13, 14, 18, 19, -2, 14, 18, 19, 21, -1, -1, -1, -1, -1, -1, -1, -1, -1, 6, 7, 8, -2, 10, 11, 12, 13, 14, 11, 12, 13, 14, 18, 19, 21, 14, 18, 19, -1]


In [173]:
chorus_range = get_chorus(chorus_start, chorus_length, lyrics)

print('Recognized Chorus: ', chorus_start, chorus_length)

Recognized Chorus:  10 16


In [174]:
chorus_range

range(10, 26)

# Retrieval and Re-ranking

In [175]:
select_notes = []  # selected_melodies
select_chords = []  # selected chords
is_chorus = 0  # is a chorus?
note_string = ''  # the 'melody context' mentioned in the paper.
bar_idx = 0  # current bar index. it is used to replace bar index in retrieved pieces.
last_note_end = -16

## Пройдёмся вручную по циклу (пройдём через каждую строчку)

In [176]:
line_num = 0

In [677]:
line_num+=1

In [678]:
print(line_num < len(sentence))

False


Если False, то заканчиваем

In [670]:
print('Line ', line_num)
print('Lyrics: ', lyrics.strip().split('[sep]')[line_num])

Line  54
Lyrics:   Good e nough, oh, -ough


In [671]:
cnt = len(sentence[line_num])
if cnt <= 2 and parent[line_num] == -2:  # if length is too short, do not partially share
    parent[line_num] = -1
    print('Changed struct item into -1')

In [672]:
is_last_sentence = (line_num == len(sentence) - 1)
if is_last_sentence:
    print('It\'s the last sentence')

It's the last sentence


In [673]:
if line_num in chorus_range:
    is_chorus = 1
    print('It is a chorus')
else:
    is_chorus = 0
    print('is not chorus')

is not chorus


In [674]:
if parent[line_num] == -1:
    print('No sharing.')

No sharing.


In [675]:
print('Struct value: ', parent[line_num])

Struct value:  -1


### -1

In [676]:
print('No sharing.')
one_syllable_multi_notes_probabilty = random.randint(1,100)
if one_syllable_multi_notes_probabilty == 1:
    is_1smn = 1
    connect_notes = random.randint(1,2)
    cnt += connect_notes
    connect_start = random.randint(1,cnt)
    print('One Syllable Multi Notes range:',connect_start, connect_start + connect_notes)

if len(select_notes) == 0:  # The first sentence of a song.
    last_bar = None
else:
    last_bar = select_notes[-1]

selected_bars = select_melody(is_maj, is_chorus, cnt, last_bar, chord, chord_ptr, is_last_sentence)
if cnt < 9 and len(selected_bars) > 0:
    selected_bars = lm_score(selected_bars, note_string, bar_idx)
    # selected_bars = no_keep_trend(selected_bars)
    bar_chord = selected_bars[random.randint(0, len(selected_bars) - 1)]
    s_bar = bar_chord[0]
    s_chord = bar_chord[1]
    s_bar, bar_idx = fill_template(s_bar,
                                    bar_idx)  # The returned bar index is the first bar index which should be in the next sentence, that is s_bar + 1.
else:  # If no pieces is retrieved or there are too many syllables in a lyric.
    print('No pieces is retrieved or there are too many syllables in a lyric. Split the lyric.')
    s_bar = ''
    s_chord = ''
    origin_cnt = cnt
    error = 0
    while cnt > 0:
        l = max(origin_cnt // 3, 5)
        r = max(origin_cnt // 2, 7)  # Better to use long pieces, for better coherency.
        split_len = random.randint(l, r)
        if split_len > cnt:
            split_len = cnt
        print('Split at ', split_len)
        selected_bars = select_melody(is_maj, is_chorus, split_len, last_bar, chord, chord_ptr,
                                        is_last_sentence)
        if len(selected_bars) > 0:
            selected_bars = lm_score(selected_bars, note_string + s_bar, bar_idx)
            bar_chord = selected_bars[random.randint(0, len(selected_bars) - 1)]
            last_bar = bar_chord[0]
            last_chord = bar_chord[1]
            s_bar = splice(s_bar, last_bar)
            s_chord += ' ' + last_chord

            # Explanation: if this condition is true, i.e., the length of s_bar + last_bar == the length of last_bar,
            # then the only possibility is that we are in the first step of this while loop. We need to replace the bar index in retrieved pieces with the true bar index.
            # In the following steps, there is no need to do so because there is a implicit 'fill_template' in 'splice'.
            if len(s_bar) == len(last_bar):
                s_bar, bar_idx = fill_template(s_bar, bar_idx)
            s_chord, chord_ptr = polish_chord(s_bar, s_chord, chord, chord_ptr)
            last_bar = s_bar
            cnt -= split_len
        else:
            error += 1
            if error >= 10:
                print('Database has not enough pieces to support ROC.')
                exit()

    s_chord = s_chord[1:]

s_bar, bar_idx, last_note_end = polish(s_bar, last_note_end)
s_chord, chord_ptr = polish_chord(s_bar, s_chord, chord, chord_ptr)
note_string += s_bar
select_notes.append(s_bar)
select_chords.append(s_chord)
print('Selected notes: ', s_bar)
print('Chords: ', s_chord)

No sharing.
Retrive melody...
Filter by pitch range...
Selected notes:  NOT bar_112 Pos_0 Pitch_52 Dur_6 NOT bar_112 Pos_6 Pitch_52 Dur_4 NOT bar_112 Pos_10 Pitch_52 Dur_2 NOT bar_112 Pos_12 Pitch_50 Dur_4 HALF bar_113 Pos_0 Pitch_48 Dur_2 
Chords:  C: C: C:


### -2

In [526]:
print('Share partial melody from the previous lyric.')

l = min(cnt // 3,
        3)  # As mentioned in 'Concatenation and Polish' Section, for adjacents lyrics having the same syllabels number,
r = min(cnt // 2, 5)  # we 'polish their melodies to sound similar'

# modify some notes then share.
replace_len = random.randint(l, r)
last_bar = ' '.join(select_notes[-1].split(' ')[:- replace_len * 5 - 1]) + ' '
tail = select_notes[-1].split(' ')[- replace_len * 5 - 1:]
last_chord = ' '.join(chord_truc(last_bar, select_chords[-1]))
selected_bars = select_melody(is_maj, is_chorus, replace_len, last_bar, chord, chord_ptr,
                                is_last_sentence)
selected_bars = lm_score(selected_bars, note_string + last_bar, bar_idx)
for bar_chord in selected_bars:
    bar = bar_chord[0]
    s_chord = bar_chord[1]
    s_bar = splice(last_bar, bar)
    if not_mono(s_bar) and not_duplicate(s_bar, select_notes[-1]):
        s_chord = last_chord + ' ' + s_chord
        break

s_bar, bar_idx = fill_template(s_bar, bar_idx)
s_bar = s_bar.split(' ')

for i in range(2, len(tail)):  # Modify duration
    if i % 5 == 2 or i % 5 == 1:  # dur and cadence
        s_bar[-i] = tail[-i]
s_bar = ' '.join(s_bar)

s_bar, bar_idx, last_note_end = polish(s_bar, last_note_end, True)
s_chord, chord_ptr = polish_chord(s_bar, s_chord, chord, chord_ptr)
note_string += s_bar
select_notes.append(s_bar)
select_chords.append(s_chord)

print('Modified notes: ', s_bar)
print('chords: ', s_chord)

Share partial melody from the previous lyric.
Retrive melody...
Filter by pitch range...
Modified notes:  NOT bar_81 Pos_0 Pitch_62 Dur_2 NOT bar_81 Pos_2 Pitch_62 Dur_2 NOT bar_81 Pos_4 Pitch_62 Dur_2 NOT bar_81 Pos_6 Pitch_66 Dur_2 NOT bar_81 Pos_8 Pitch_71 Dur_2 HALF bar_81 Pos_10 Pitch_66 Dur_2 
chords:  G: B:m


### else

In [667]:
# 'struct value is postive' as mentioned in the paper, we directly share melodies.
print('Share notes with sentence No.', parent[line_num])

s_bar = copy.deepcopy(select_notes[parent[line_num]])
s_chord = copy.deepcopy(select_chords[parent[line_num]])
s_bar, bar_idx = fill_template(s_bar, bar_idx)
s_bar, bar_idx, last_note_end = polish(s_bar, last_note_end, True)
s_chord, chord_ptr = polish_chord(s_bar, s_chord, chord, chord_ptr)
note_string += s_bar
select_notes.append(s_bar)
select_chords.append(s_chord)

Share notes with sentence No. 19


### Если struct value = -1 (True)

In [26]:
# Будет ли слог с несколькими нотами?
one_syllable_multi_notes_probabilty = random.randint(1,100)
if one_syllable_multi_notes_probabilty == 1:
    print('One syllable will have several notes')
    is_1smn = 1
    connect_notes = random.randint(1,2)
    cnt += connect_notes
    connect_start = random.randint(1,cnt)
    print('One Syllable Multi Notes range:',connect_start, connect_start + connect_notes)

In [27]:
if len(select_notes) == 0:  # The first sentence of a song.
    last_bar = None
    print('Это первая строчка в песне')
else:
    last_bar = select_notes[-1]
    print('Last bar: ', last_bar)

Это первая строчка в песне


In [28]:
selected_bars = select_melody(is_maj, is_chorus, cnt, last_bar, chord, chord_ptr, is_last_sentence)
print(selected_bars[0])

Retrive melody...
Start a song...
('NOT bar_X Pos_4 Pitch_64 Dur_1 NOT bar_X Pos_5 Pitch_64 Dur_1 NOT bar_X Pos_7 Pitch_64 Dur_4 NOT bar_X Pos_11 Pitch_62 Dur_1 NOT bar_X Pos_12 Pitch_60 Dur_2 NOT bar_X Pos_14 Pitch_62 Dur_2 NOT bar_Y Pos_0 Pitch_64 Dur_2 NOT bar_Y Pos_2 Pitch_64 Dur_1 NOT bar_Y Pos_3 Pitch_64 Dur_3 NOT bar_Y Pos_6 Pitch_64 Dur_1 NOT bar_Y Pos_7 Pitch_64 Dur_4 NOT bar_Y Pos_11 Pitch_62 Dur_1 NOT bar_Y Pos_12 Pitch_60 Dur_2 NOT bar_Y Pos_14 Pitch_62 Dur_2 ', 'C: C: C: C: C:')


In [29]:
if cnt < 9 and len(selected_bars) > 0:
    print('Количество слогов в строчке меньше 9')

Если есть вывод, то выполнить:

In [1320]:
selected_bars = lm_score(selected_bars, note_string, bar_idx)
# selected_bars = no_keep_trend(selected_bars)
bar_chord = selected_bars[random.randint(0, len(selected_bars) - 1)]
s_bar = bar_chord[0]
s_chord = bar_chord[1]
s_bar, bar_idx = fill_template(s_bar, bar_idx) 
# The returned bar index is the first bar index which should be in the next sentence, that is s_bar + 1

Если нет: т. е. количество слогов в строчке большое, и/или не было найдено подходящих мелодий-кусочков

In [30]:
print('No pieces is retrieved or there are too many syllables in a lyric. Split the lyric.')

s_bar = ''
s_chord = ''
origin_cnt = cnt
error = 0

No pieces is retrieved or there are too many syllables in a lyric. Split the lyric.


In [31]:
print('Длина строчки: ', origin_cnt)

Длина строчки:  14


#### делается пока cnt > 0: 

In [32]:
cnt > 0

True

In [53]:
# Уменьшим лину строчки
l = max(origin_cnt // 3, 5)
r = max(origin_cnt // 2, 7)  # Better to use long pieces, for better coherency.
split_len = random.randint(l, r)

In [54]:
if split_len > cnt:
    split_len = cnt
print('Split at ', split_len)

Split at  4


In [55]:
selected_bars = select_melody(is_maj, is_chorus, split_len, last_bar, chord, chord_ptr, is_last_sentence)
print(len(selected_bars), ' melodies retrieved')

Retrive melody...
Filter by pitch range...
61  melodies retrieved


Если мелодий не нашлось, то:

In [360]:
error += 1
if error >= 10:
    print('Database has not enough pieces to support ROC.')

Если хотя бы сколько-то мелодий нашлось, выполнить:

In [56]:
selected_bars = lm_score(selected_bars, note_string + s_bar, bar_idx)
len(selected_bars)

16

In [57]:
bar_chord = selected_bars[random.randint(0, len(selected_bars) - 1)]
last_bar = bar_chord[0]
last_chord = bar_chord[1]
print('Last bar: ', last_bar)
print('Last chord: ', last_chord)

Last bar:  NOT bar_X Pos_8 Pitch_64 Dur_2 NOT bar_X Pos_10 Pitch_67 Dur_2 NOT bar_X Pos_12 Pitch_69 Dur_2 NOT bar_X Pos_14 Pitch_71 Dur_2 
Last chord:  E:m E:m E:m


In [58]:
# Соединяем полученную мелодию с предыдущей сохраняя все правила
s_bar = splice(s_bar, last_bar)
s_bar

'NOT bar_0 Pos_4 Pitch_64 Dur_2 NOT bar_0 Pos_6 Pitch_64 Dur_2 NOT bar_0 Pos_8 Pitch_64 Dur_4 NOT bar_0 Pos_12 Pitch_62 Dur_2 AUT bar_0 Pos_14 Pitch_60 Dur_8 NOT bar_0 Pos_6 Pitch_67 Dur_2 NOT bar_0 Pos_8 Pitch_67 Dur_2 NOT bar_0 Pos_10 Pitch_69 Dur_2 NOT bar_0 Pos_12 Pitch_71 Dur_2 NOT bar_0 Pos_14 Pitch_67 Dur_2 NOT bar_1 Pos_0 Pitch_64 Dur_2 NOT bar_1 Pos_2 Pitch_67 Dur_2 NOT bar_1 Pos_4 Pitch_69 Dur_2 NOT bar_1 Pos_6 Pitch_71 Dur_2 '

In [59]:
# Соединяем текущие аккорды с предыдущими
s_chord += ' ' + last_chord
print(s_chord)

 C: E:m E:m E:m


In [60]:
# Explanation: if this condition is true, i.e., the length of s_bar + last_bar == the length of last_bar,
# then the only possibility is that we are in the first step of this while loop. 
# We need to replace the bar index in retrieved pieces with the true bar index.
# In the following steps, there is no need to do so because there is a implicit 'fill_template' in 'splice'.
if len(s_bar) == len(last_bar):
    s_bar, bar_idx = fill_template(s_bar, bar_idx)

In [61]:
s_chord, chord_ptr = polish_chord(s_bar, s_chord, chord, chord_ptr)
last_bar = s_bar
cnt -= split_len

In [62]:
cnt

0

#### как только перестанет выполняться cnt > 0: 

In [63]:
s_chord = s_chord[1:]

In [64]:
s_chord

'C: E:m'

#### продолжаем работать с полученной мелодией

In [65]:
s_bar, bar_idx, last_note_end = polish(s_bar, last_note_end)

In [66]:
s_chord, chord_ptr = polish_chord(s_bar, s_chord, chord, chord_ptr)
print(s_chord)
chord_ptr

C: C: E:m E:m


2

In [67]:
note_string += s_bar
select_notes.append(s_bar)
select_chords.append(s_chord)
print('Selected notes: ', s_bar)
print('Chords: ', s_chord)

Selected notes:  NOT bar_0 Pos_0 Pitch_64 Dur_2 NOT bar_0 Pos_2 Pitch_64 Dur_2 NOT bar_0 Pos_4 Pitch_64 Dur_4 NOT bar_0 Pos_8 Pitch_62 Dur_2 NOT bar_0 Pos_10 Pitch_60 Dur_8 NOT bar_1 Pos_2 Pitch_67 Dur_2 NOT bar_1 Pos_4 Pitch_67 Dur_2 NOT bar_1 Pos_6 Pitch_69 Dur_2 NOT bar_1 Pos_8 Pitch_71 Dur_2 NOT bar_1 Pos_10 Pitch_67 Dur_2 NOT bar_1 Pos_12 Pitch_64 Dur_2 NOT bar_1 Pos_14 Pitch_67 Dur_2 NOT bar_2 Pos_0 Pitch_69 Dur_2 HALF bar_2 Pos_2 Pitch_71 Dur_2 
Chords:  C: C: E:m E:m


### Если struct value = -2 (True)

In [1142]:
print('Share partial melody from the previous lyric.')
l = min(cnt // 3, 3)  
# As mentioned in 'Concatenation and Polish' Section, for adjacents lyrics having the same syllabels number,
r = min(cnt // 2, 5)  
# we 'polish their melodies to sound similar'
print('l = ', l, ' r = ', r)

Share partial melody from the previous lyric.
l =  2  r =  3


In [1143]:
# modify some notes then share.
replace_len = random.randint(l, r)
print('replace length: ', replace_len)
last_bar = ' '.join(select_notes[-1].split(' ')[:- replace_len * 5 - 1]) + ' '
print(last_bar)

replace length:  3
NOT bar_90 Pos_0 Pitch_55 Dur_2 NOT bar_90 Pos_2 Pitch_55 Dur_2 NOT bar_90 Pos_4 Pitch_55 Dur_2 


In [1144]:
tail = select_notes[-1].split(' ')[- replace_len * 5 - 1:]
last_chord = ' '.join(chord_truc(last_bar, select_chords[-1]))
selected_bars = select_melody(is_maj, is_chorus, replace_len, last_bar, chord, chord_ptr, is_last_sentence)

print('last chord: ', last_chord)

Retrive melody...
Filter by pitch range...
last chord:  C:


In [1145]:
selected_bars = lm_score(selected_bars, note_string + last_bar, bar_idx)
selected_bars

[('NOT bar_X Pos_10 Pitch_55 Dur_2 NOT bar_X Pos_12 Pitch_57 Dur_2 NOT bar_X Pos_14 Pitch_59 Dur_2 ',
  'E:m E:m E:m'),
 ('NOT bar_X Pos_8 Pitch_55 Dur_2 NOT bar_X Pos_10 Pitch_57 Dur_2 NOT bar_X Pos_12 Pitch_59 Dur_4 ',
  'E:m E:m E:m'),
 ('NOT bar_X Pos_10 Pitch_55 Dur_2 NOT bar_X Pos_12 Pitch_55 Dur_2 NOT bar_X Pos_14 Pitch_59 Dur_4 ',
  'E:m E:m E:m'),
 ('NOT bar_X Pos_0 Pitch_59 Dur_4 NOT bar_X Pos_4 Pitch_55 Dur_2 AUT bar_X Pos_6 Pitch_55 Dur_6 ',
  'E:m E:m E:m'),
 ('NOT bar_X Pos_0 Pitch_59 Dur_4 NOT bar_X Pos_4 Pitch_57 Dur_2 AUT bar_X Pos_6 Pitch_55 Dur_6 ',
  'E:m E:m E:m'),
 ('NOT bar_X Pos_10 Pitch_52 Dur_2 NOT bar_X Pos_12 Pitch_57 Dur_2 NOT bar_X Pos_14 Pitch_59 Dur_2 ',
  'E:m E:m E:m'),
 ('NOT bar_X Pos_11 Pitch_55 Dur_2 NOT bar_X Pos_13 Pitch_57 Dur_2 NOT bar_X Pos_15 Pitch_59 Dur_2 ',
  'E:m E:m E:m'),
 ('NOT bar_X Pos_12 Pitch_55 Dur_2 NOT bar_X Pos_14 Pitch_57 Dur_1 NOT bar_X Pos_15 Pitch_59 Dur_1 ',
  'E:m E:m E:m'),
 ('NOT bar_X Pos_0 Pitch_57 Dur_2 NOT bar_X Pos

#### for bar chord in selected bars

In [1146]:
j = 0

In [1147]:
j == len(selected_bars)

False

In [1148]:
bar_chord = selected_bars[j]
bar = bar_chord[0]
s_chord = bar_chord[1]
print(bar_chord)
print(s_chord)

('NOT bar_X Pos_10 Pitch_55 Dur_2 NOT bar_X Pos_12 Pitch_57 Dur_2 NOT bar_X Pos_14 Pitch_59 Dur_2 ', 'E:m E:m E:m')
E:m E:m E:m


In [1149]:
s_bar = splice(last_bar, bar)
s_bar

'NOT bar_90 Pos_0 Pitch_55 Dur_2 NOT bar_90 Pos_2 Pitch_55 Dur_2 NOT bar_90 Pos_4 Pitch_55 Dur_2 NOT bar_90 Pos_6 Pitch_55 Dur_2 NOT bar_90 Pos_8 Pitch_57 Dur_2 NOT bar_90 Pos_10 Pitch_59 Dur_2 '

In [1150]:
select_notes[-1]

'NOT bar_90 Pos_0 Pitch_55 Dur_2 NOT bar_90 Pos_2 Pitch_55 Dur_2 NOT bar_90 Pos_4 Pitch_55 Dur_2 NOT bar_90 Pos_6 Pitch_52 Dur_2 NOT bar_90 Pos_8 Pitch_50 Dur_2 HALF bar_90 Pos_10 Pitch_48 Dur_2 '

In [1151]:
if not_mono(s_bar) and not_duplicate(s_bar, select_notes[-1]):
    s_chord = last_chord + ' ' + s_chord
    print('The melody is not monotonous and doesn\'t duplicate the previous one')

The melody is not monotonous and doesn't duplicate the previous one


Если вывод есть, то выход из цикла

In [ ]:
j += 1

#### продолжение

In [1152]:
s_bar, bar_idx = fill_template(s_bar, bar_idx)
s_bar = s_bar.split(' ')

In [1153]:
for k in range(2, len(tail)):  # Modify duration
    if k % 5 == 2 or k % 5 == 1:  # dur and cadence
        s_bar[-k] = tail[-k]             

In [1154]:
s_bar = ' '.join(s_bar)

In [1155]:
s_bar, bar_idx, last_note_end = polish(s_bar, last_note_end, True) # melody polishing
s_chord, chord_ptr = polish_chord(s_bar, s_chord, chord, chord_ptr) # chord polishing

In [1156]:
note_string += s_bar
select_notes.append(s_bar)
select_chords.append(s_chord)

print('Modified notes: ', s_bar)
print('chords: ', s_chord)

Modified notes:  NOT bar_91 Pos_0 Pitch_55 Dur_2 NOT bar_91 Pos_2 Pitch_55 Dur_2 NOT bar_91 Pos_4 Pitch_55 Dur_2 NOT bar_91 Pos_6 Pitch_55 Dur_2 NOT bar_91 Pos_8 Pitch_57 Dur_2 HALF bar_91 Pos_10 Pitch_59 Dur_2 
chords:  C: E:m


### Если не -1 и не -2

In [1306]:
# 'struct value is postive' as mentioned in the paper, we directly share melodies.
print('Share notes with sentence No.', parent[i])

Share notes with sentence No. 19


In [1307]:
s_bar = copy.deepcopy(select_notes[parent[i]])
s_chord = copy.deepcopy(select_chords[parent[i]])

s_bar, bar_idx = fill_template(s_bar, bar_idx)
s_bar, bar_idx, last_note_end = polish(s_bar, last_note_end, True)
s_chord, chord_ptr = polish_chord(s_bar, s_chord, chord, chord_ptr)

note_string += s_bar
select_notes.append(s_bar)
select_chords.append(s_chord)

### Мелодия сделана

# Быстрый код

In [6]:
is_debug = True

In [7]:
chord_ptr = 0

is_maj = 1
#is_maj = 0


sentence = lyrics.strip().split('[sep]')
name = sentence[0]
sentence = [len(x.strip().split(' ')) * '_' for x in sentence] # parsing the syllables

# if use_sentiment:
#     sent = '.'.join(sentence)
#     blob = TextBlob(sent)
#     polarity = 0
#     for s in blob.sentences:
#         polarity += s.sentiment.polarity
#     if polarity >= 0:
#         is_maj = 1
#     else:
#         is_maj = 0

print('Tonality:', is_maj)

# structure recognition
parent, chorus_start, chorus_length = Lyrics_match(
    sentence)  # The last element must be -1, because the chord should go back to tonic
if is_debug:
    print('Struct Array: ', parent)

chorus_range = get_chorus(chorus_start, chorus_length, lyrics)
if is_debug:
    print('Recognized Chorus: ', chorus_start, chorus_length)

select_notes = []  # selected_melodies
select_chords = []  # selected chords
is_chorus = 0  # is a chorus?
note_string = ''  # the 'melody context' mentioned in the paper.
bar_idx = 0  # current bar index. it is used to replace bar index in retrieved pieces.
last_note_end = -16
# is_1smn = 0             # Does 1 Syllable align with Multi Notes? In the future, we will explore better methods to realize this function. Here by default, we disable it.

for i in range(len(sentence)):

    print('Lyrics: ', lyrics.strip().split('[sep]')[i])

    is_last_sentence = (i == len(sentence) - 1)

    if i in chorus_range:
        is_chorus = 1
    else:
        is_chorus = 0

    cnt = len(sentence[i])
    if cnt <= 2 and parent[i] == -2:  # if length is too short, do not partially share
        parent[i] = -1

    # Following codes correspond to 'Retrieval and Re-ranking' in Section 3.2.
    # parent[i] is the 'struct value' in the paper.
    if parent[i] == -1:
        if is_debug:
            print('No sharing.')
        one_syllable_multi_notes_probabilty = random.randint(1,100)
        if one_syllable_multi_notes_probabilty == 1:
            is_1smn = 1
            connect_notes = random.randint(1,2)
            cnt += connect_notes
            connect_start = random.randint(1,cnt)
            print('One Syllable Multi Notes range:',connect_start, connect_start + connect_notes)

        if len(select_notes) == 0:  # The first sentence of a song.
            last_bar = None
        else:
            last_bar = select_notes[-1]

        selected_bars = select_melody(is_maj, is_chorus, cnt, last_bar, chord, chord_ptr, is_last_sentence)
        if cnt < 9 and len(selected_bars) > 0:
            selected_bars = lm_score(selected_bars, note_string, bar_idx)
            # selected_bars = no_keep_trend(selected_bars)
            bar_chord = selected_bars[random.randint(0, len(selected_bars) - 1)]
            s_bar = bar_chord[0]
            s_chord = bar_chord[1]
            s_bar, bar_idx = fill_template(s_bar,
                                            bar_idx)  # The returned bar index is the first bar index which should be in the next sentence, that is s_bar + 1.
        else:  # If no pieces is retrieved or there are too many syllables in a lyric.
            if is_debug:
                print('No pieces is retrieved or there are too many syllables in a lyric. Split the lyric.')
            s_bar = ''
            s_chord = ''
            origin_cnt = cnt
            error = 0
            while cnt > 0:
                l = max(origin_cnt // 3, 5)
                r = max(origin_cnt // 2, 7)  # Better to use long pieces, for better coherency.
                split_len = random.randint(l, r)
                if split_len > cnt:
                    split_len = cnt
                if is_debug:
                    print('Split at ', split_len)
                selected_bars = select_melody(is_maj, is_chorus, split_len, last_bar, chord, chord_ptr,
                                                is_last_sentence)
                if len(selected_bars) > 0:
                    selected_bars = lm_score(selected_bars, note_string + s_bar, bar_idx)
                    bar_chord = selected_bars[random.randint(0, len(selected_bars) - 1)]
                    last_bar = bar_chord[0]
                    last_chord = bar_chord[1]
                    s_bar = splice(s_bar, last_bar)
                    s_chord += ' ' + last_chord

                    # Explanation: if this condition is true, i.e., the length of s_bar + last_bar == the length of last_bar,
                    # then the only possibility is that we are in the first step of this while loop. We need to replace the bar index in retrieved pieces with the true bar index.
                    # In the following steps, there is no need to do so because there is a implicit 'fill_template' in 'splice'.
                    if len(s_bar) == len(last_bar):
                        s_bar, bar_idx = fill_template(s_bar, bar_idx)
                    s_chord, chord_ptr = polish_chord(s_bar, s_chord, chord, chord_ptr)
                    last_bar = s_bar
                    cnt -= split_len
                else:
                    error += 1
                    if error >= 10:
                        print('Database has not enough pieces to support ROC.')
                        exit()

            s_chord = s_chord[1:]

        s_bar, bar_idx, last_note_end = polish(s_bar, last_note_end)
        s_chord, chord_ptr = polish_chord(s_bar, s_chord, chord, chord_ptr)
        note_string += s_bar
        select_notes.append(s_bar)
        select_chords.append(s_chord)
        if is_debug:
            print('Selected notes: ', s_bar)
            print('Chords: ', s_chord)

    elif parent[i] == -2:
        if is_debug:
            print('Share partial melody from the previous lyric.')

        l = min(cnt // 3,
                3)  # As mentioned in 'Concatenation and Polish' Section, for adjacents lyrics having the same syllabels number,
        r = min(cnt // 2, 5)  # we 'polish their melodies to sound similar'

        # modify some notes then share.
        replace_len = random.randint(l, r)
        last_bar = ' '.join(select_notes[-1].split(' ')[:- replace_len * 5 - 1]) + ' '
        tail = select_notes[-1].split(' ')[- replace_len * 5 - 1:]
        last_chord = ' '.join(chord_truc(last_bar, select_chords[-1]))
        selected_bars = select_melody(is_maj, is_chorus, replace_len, last_bar, chord, chord_ptr,
                                        is_last_sentence)
        selected_bars = lm_score(selected_bars, note_string + last_bar, bar_idx)
        for bar_chord in selected_bars:
            bar = bar_chord[0]
            s_chord = bar_chord[1]
            s_bar = splice(last_bar, bar)
            if not_mono(s_bar) and not_duplicate(s_bar, select_notes[-1]):
                s_chord = last_chord + ' ' + s_chord
                break

        s_bar, bar_idx = fill_template(s_bar, bar_idx)
        s_bar = s_bar.split(' ')

        for i in range(2, len(tail)):  # Modify duration
            if i % 5 == 2 or i % 5 == 1:  # dur and cadence
                s_bar[-i] = tail[-i]
        s_bar = ' '.join(s_bar)

        s_bar, bar_idx, last_note_end = polish(s_bar, last_note_end, True)
        s_chord, chord_ptr = polish_chord(s_bar, s_chord, chord, chord_ptr)
        note_string += s_bar
        select_notes.append(s_bar)
        select_chords.append(s_chord)

        if is_debug:
            print('Modified notes: ', s_bar)
            print('chords: ', s_chord)
    else:
        # 'struct value is postive' as mentioned in the paper, we directly share melodies.
        if is_debug:
            print('Share notes with sentence No.', parent[i])

        s_bar = copy.deepcopy(select_notes[parent[i]])
        s_chord = copy.deepcopy(select_chords[parent[i]])
        s_bar, bar_idx = fill_template(s_bar, bar_idx)
        s_bar, bar_idx, last_note_end = polish(s_bar, last_note_end, True)
        s_chord, chord_ptr = polish_chord(s_bar, s_chord, chord, chord_ptr)
        note_string += s_bar
        select_notes.append(s_bar)
        select_chords.append(s_chord)

    if is_debug:
        print(
            '----------------------------------------------------------------------------------------------------------')

if is_debug:
    print(select_chords)
    print(select_notes)

Tonality: 1
for test: [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
length: 26
Struct Array:  [-1, -1, -1, -1, -1, -1, -1, -1, 7, 8, -2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
Recognized Chorus:  -1 2
Lyrics:  Кто зна ет, за чем за жи га ют звёз ды? 
No sharing.
Retrive melody...
Start a song...
No pieces is retrieved or there are too many syllables in a lyric. Split the lyric.
Split at  7
Retrive melody...
Start a song...
Split at  4
Retrive melody...
Filter by pitch range...
Selected notes:  NOT bar_0 Pos_0 Pitch_64 Dur_2 NOT bar_0 Pos_2 Pitch_64 Dur_2 NOT bar_0 Pos_4 Pitch_64 Dur_3 NOT bar_0 Pos_7 Pitch_62 Dur_2 NOT bar_0 Pos_9 Pitch_60 Dur_3 NOT bar_0 Pos_12 Pitch_62 Dur_2 NOT bar_0 Pos_14 Pitch_64 Dur_3 NOT bar_1 Pos_1 Pitch_64 Dur_2 NOT bar_1 Pos_3 Pitch_62 Dur_5 NOT bar_1 Pos_8 Pitch_55 Dur_2 HALF bar_1 Pos_10 Pitch_55 Dur_2 
Chords:  C: C:
------------------------------------------------------------

# Конечный результат

In [8]:
print(select_chords)

['C: C:', 'G: A:m A:m F: F:', 'C: C: G: G: G:', 'A:m A:m F: F: F:', 'C: C: G:', 'A:m F: F: F:', 'C: C: G: G: G:', 'A:m F: F: F: F:', 'A:m F: F: F: F:', 'A:m F: F: F: F:', 'A:m F: C: C: C:', 'G: G: A:m A:m A:m', 'F: F: C: C: C:', ' ', 'C: C: C:', 'G: G: A:m A:m', 'F: F: C: C:', 'G: G: A:m', 'F: F: C: C: C:', 'G: G: A:m A:m A:m', 'F: F: C: C: C:', 'G: G: A:m A:m A:m', 'F: F: F: F: C: C: G: G: G:', 'A:m A:m A:m A:m F:', 'C: C: G: G:', 'A:m A:m C:']


In [9]:
print(select_notes[:1])

['NOT bar_0 Pos_0 Pitch_64 Dur_2 NOT bar_0 Pos_2 Pitch_64 Dur_2 NOT bar_0 Pos_4 Pitch_64 Dur_3 NOT bar_0 Pos_7 Pitch_62 Dur_2 NOT bar_0 Pos_9 Pitch_60 Dur_3 NOT bar_0 Pos_12 Pitch_62 Dur_2 NOT bar_0 Pos_14 Pitch_64 Dur_3 NOT bar_1 Pos_1 Pitch_64 Dur_2 NOT bar_1 Pos_3 Pitch_62 Dur_5 NOT bar_1 Pos_8 Pitch_55 Dur_2 HALF bar_1 Pos_10 Pitch_55 Dur_2 ']


In [1415]:
save_demo(
    note_string, 
    select_chords, 
    name, 
    'en',
    lyrics.strip().split('[sep]'),
    [len(i) for i in sentence])

Save the melody to take 1.mid.mid


NameError: name 'is_debug' is not defined

In [10]:
conn.close()

# Select melody: how does it work

In [72]:
selected_bars = select_melody(is_maj, is_chorus, cnt, last_bar, chord, chord_ptr, is_last_sentence)
print(selected_bars)

NameError: name 'c' is not defined

In [73]:
length = cnt
print('Length = ', length)

Length =  14


In [74]:
# Поиск в базе кусочка по длине, ладу и тому, является ли припевом или нет
cursor = c.execute(
    "SELECT DISTINCT NOTES, CHORDS from MELOLIB  where LENGTH = '{}' and CHORUS = '{}' and MAJOR = '{}' ".format(
        length, is_chorus, is_maj
        )
        )  # and MAJOR = '{}'

In [75]:
candidates_bars = []
print("Retrive melody...")
for row in cursor:
    notes = row[0]
    cd_ = row[1]
    candidates_bars.append((notes, cd_))

print(candidates_bars[0])
print(len(candidates_bars), ' melodies retirved')

Retrive melody...
('NOT bar_X Pos_2 Pitch_76 Dur_1 NOT bar_X Pos_4 Pitch_76 Dur_1 NOT bar_X Pos_6 Pitch_76 Dur_1 NOT bar_X Pos_8 Pitch_76 Dur_2 NOT bar_X Pos_10 Pitch_74 Dur_2 NOT bar_X Pos_12 Pitch_72 Dur_2 NOT bar_X Pos_14 Pitch_74 Dur_4 NOT bar_Y Pos_2 Pitch_72 Dur_1 NOT bar_Y Pos_4 Pitch_72 Dur_1 NOT bar_Y Pos_6 Pitch_72 Dur_1 NOT bar_Y Pos_8 Pitch_72 Dur_2 NOT bar_Y Pos_10 Pitch_71 Dur_2 NOT bar_Y Pos_12 Pitch_69 Dur_2 NOT bar_Y Pos_14 Pitch_67 Dur_4 ', 'C: C: D:m7 D:m7 C:maj7')
2089  melodies retirved


In [76]:
chord_ptr

0

In [77]:
# Filter by chords.
chord_list_ = chord.strip().split(' ')
chord_list_ = chord_list_[chord_ptr:] + chord_list_[:chord_ptr]
re_str = ''
print(chord_list_)

['C:', 'E:m', 'G:', 'B:m', 'C:', 'E:m', 'G:', 'B:m', 'C:', 'E:m', 'G:', 'B:m', 'C:', 'E:m', 'G:', 'B:m', 'C:', 'E:m', 'G:', 'B:m', 'C:', 'E:m', 'G:', 'B:m', 'C:', 'E:m', 'G:', 'B:m', 'C:', 'E:m', 'G:', 'B:m', 'C:', 'E:m', 'G:', 'B:m', 'C:', 'E:m', 'G:', 'B:m', 'C:', 'E:m', 'G:', 'B:m', 'C:', 'E:m', 'G:', 'B:m', 'C:', 'E:m', 'G:', 'B:m', 'C:']


In [78]:
# Здесь задано условие, что на последнем такте должна быть тоника

if not is_last_sentence:
    key = ''
else:
    if is_maj:
        key = ' C:'
    else:
        key = ' A:m'

In [79]:
# Оформление строки для запроса
    # For the given chord progression, we generate a regex like:
    # A:m F: G: C: -> ^A:m( A:m)*( F:)+( G:)+( C:)*$|^A:m( A:m)*( F:)+( G:)*$|^A:m( A:m)*( F:)*$|^A:m( A:m)*$
    # Given the regex, we find matched pieces.
    # We design the regex like this because alternations in regular expressions are evaluated from left to right,
    # the piece with the most various chords will be selected, if there's any.
for j in range(len(chord_list_), 0, -1):
    re_str += '^({}( {})*'.format(chord_list_[0], chord_list_[0])
    for idx in range(1, j):
        re_str += '( {})+'.format(chord_list_[idx])
    re_str = re_str[:-1]
    re_str += '*{})$|'.format(key)
re_str = re_str[:-1]


In [80]:
# Фльтр по аккордам
tmp_candidates = []
for row in candidates_bars:
    if re.match(r'{}'.format(re_str), row[1]):
        tmp_candidates.append(row)

print(len(tmp_candidates), ' candidates remain')

39  candidates remain


In [81]:
# Что делать, если кандидатов не найдено
if len(tmp_candidates) == 0:
    print('меняем запрос')
    re_str = '^{}( {})*$'.format(chord_list_[-1], chord_list_[-1])
    for row in candidates_bars:
        if re.match(r'{}'.format(re_str), row[1]):
            tmp_candidates.append(row)
    print(len(tmp_candidates), ' candidates remain')

In [82]:
# Также что делать, если кандидатов не нашлось
if len(tmp_candidates) > 0:
    candidates_bars = tmp_candidates
else:
    if is_maj:
        re_str = '^C:( C:)*$'
    else:
        re_str = '^A:m( A:m)*$'
    for row in candidates_bars:
        if re.match(r'{}'.format(re_str), row[1]):
            tmp_candidates.append(row)
    if len(tmp_candidates) > 0:
        candidates_bars = tmp_candidates

print(len(candidates_bars), ' candidates remain')

39  candidates remain


In [83]:
candidates_cnt = len(candidates_bars)
if candidates_cnt == 0:
    print('No Matched Rhythm as {}'.format(length))
    selected_bars = []

Если только что был вывод, то на этом всё.

In [84]:
if last_bar == None:  # we are at the begining of a song, random select bars.
    print('Start a song...')

Start a song...


### Если это начало песни, выполнить:

In [85]:
# Фльтр слишком низких и слшком высоких нот (От pitch 57 до 66)
tmp = []
counter = 0
for bar in candidates_bars:
    notes = bar[0].split(' ')[:-1][3::5]
    notes = [int(x[6:]) for x in notes]
    for i in notes:
        if 57 > i or i > 66:
            too_high = True
            counter += 1
    too_high = False

    if not too_high:
        tmp.append(bar)

selected_bars = tmp
print(counter, ' notes filtered')
print(selected_bars[0])


349  notes filtered
('NOT bar_X Pos_2 Pitch_76 Dur_1 NOT bar_X Pos_3 Pitch_76 Dur_1 NOT bar_X Pos_4 Pitch_76 Dur_1 NOT bar_X Pos_5 Pitch_76 Dur_1 NOT bar_X Pos_6 Pitch_76 Dur_1 NOT bar_X Pos_7 Pitch_76 Dur_1 NOT bar_X Pos_8 Pitch_76 Dur_1 NOT bar_X Pos_9 Pitch_74 Dur_1 AUT bar_X Pos_10 Pitch_72 Dur_11 NOT bar_Y Pos_6 Pitch_76 Dur_1 NOT bar_Y Pos_7 Pitch_76 Dur_1 NOT bar_Y Pos_8 Pitch_76 Dur_1 NOT bar_Y Pos_9 Pitch_74 Dur_1 AUT bar_Y Pos_10 Pitch_72 Dur_11 ', 'C: C: C: C: C:')


На этом всё

### Если это не начало песни

In [ ]:
last_note = int(last_bar.split(' ')[-3][6:])
# tendency
selected_bars = []
prefer_note = None

In [ ]:
#Дальнейшее построение мелодии в зависимости от предыдущей ноты
# Major C
if is_maj:
    if last_note % 12 == 2 or last_note % 12 == 9:
        prefer_note = last_note - 2
    elif last_note % 12 == 5:
        prefer_note = last_note - 1
    elif last_note % 12 == 11:
        prefer_note = last_note + 1
# Minor A
else:
    if last_note % 12 == 11 or last_note % 12 == 2:  # 2 -> 1, 4 -> 3
        prefer_note = last_note - 2
    elif last_note % 12 == 6:  # 6 -> 5
        prefer_note = last_note - 1
    elif last_note % 12 == 7:  # 7-> 1
        prefer_note = last_note + 2

In [ ]:
if prefer_note is not None:
    for x in candidates_bars:
        if x[0][0] == prefer_note:
            selected_bars.append(x)

In [ ]:
# Добавление кусочка к кандидату
if len(selected_bars) > 0:
    print('Filter by tendency...')
    candidates_bars = selected_bars

In [ ]:
selected_bars = []
for bar in candidates_bars:
    first_pitch = int(bar[0].split(' ')[3][6:])
    if (first_pitch > last_note - 8 and first_pitch < last_note + 8):
            selected_bars.append(bar)
if len(selected_bars) > 0:
    print('Filter by pitch range...')
    selected_bars

### Если ничего не нашлось

In [ ]:
# No candidates yet? randomly return some.
print("Randomly selected...")

# LM score: closer look

In [ ]:
selected_bars = lm_score(selected_bars, note_string + s_bar, bar_idx)

In [117]:
bars = selected_bars
note_string = note_string + s_bar
print('note string: ', note_string)
print('bar index: ', bar_idx)

note string:  NOT bar_X Pos_0 Pitch_60 Dur_2 NOT bar_X Pos_2 Pitch_62 Dur_2 HALF bar_X Pos_4 Pitch_64 Dur_6 NOT bar_X Pos_10 Pitch_60 Dur_2 NOT bar_X Pos_12 Pitch_64 Dur_2 NOT bar_X Pos_14 Pitch_62 Dur_1 NOT bar_X Pos_15 Pitch_64 Dur_1 
bar index:  0


In [119]:
tmp_string = []
n = ' '.join(note_string.split(' ')[-100:])
print(n)
for sbar in bars:
    sbar_, _ = fill_template(sbar[0], bar_idx)
    tmp_string.append(n + sbar_)

print(tmp_string[0])


NOT bar_X Pos_0 Pitch_60 Dur_2 NOT bar_X Pos_2 Pitch_62 Dur_2 HALF bar_X Pos_4 Pitch_64 Dur_6 NOT bar_X Pos_10 Pitch_60 Dur_2 NOT bar_X Pos_12 Pitch_64 Dur_2 NOT bar_X Pos_14 Pitch_62 Dur_1 NOT bar_X Pos_15 Pitch_64 Dur_1 
NOT bar_X Pos_0 Pitch_60 Dur_2 NOT bar_X Pos_2 Pitch_62 Dur_2 HALF bar_X Pos_4 Pitch_64 Dur_6 NOT bar_X Pos_10 Pitch_60 Dur_2 NOT bar_X Pos_12 Pitch_64 Dur_2 NOT bar_X Pos_14 Pitch_62 Dur_1 NOT bar_X Pos_15 Pitch_64 Dur_1 NOT bar_0 Pos_0 Pitch_60 Dur_2 NOT bar_0 Pos_2 Pitch_62 Dur_2 HALF bar_0 Pos_4 Pitch_64 Dur_6 NOT bar_0 Pos_10 Pitch_60 Dur_2 NOT bar_0 Pos_12 Pitch_64 Dur_2 NOT bar_0 Pos_14 Pitch_62 Dur_1 NOT bar_0 Pos_15 Pitch_64 Dur_1 


Transformer Language model

In [121]:
custom_lm = TransformerLanguageModel.from_pretrained('music-ckps/', 'checkpoint_best.pt', tokenizer='space', batch_size=8192).cuda() ##batch_size=8192

2022-12-02 16:20:14 | INFO | fairseq.file_utils | loading archive file music-ckps/
2022-12-02 16:20:15 | INFO | fairseq.tasks.language_modeling | dictionary: 336 types
2022-12-02 16:20:15 | INFO | fairseq.models.fairseq_model | Namespace(no_progress_bar=False, log_interval=100, log_format=None, tensorboard_logdir=None, seed=1, cpu=False, tpu=False, bf16=False, memory_efficient_bf16=False, fp16=True, memory_efficient_fp16=False, fp16_no_flatten_grads=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, min_loss_scale=0.0001, threshold_loss_scale=None, user_dir=None, empty_cache_freq=0, all_gather_list_size=16384, model_parallel_size=1, checkpoint_suffix='', checkpoint_shard_count=1, quantization_config_path=None, profile=False, criterion='cross_entropy', tokenizer='space', bpe=None, optimizer='adam', lr_scheduler='inverse_sqrt', scoring='bleu', task='language_modeling', num_workers=1, skip_invalid_size_inputs_valid_test=False, max_tokens=None, batch_size=8192, r

In [122]:
# Расчёт score для каждого кусочка мелодии
score = [x['score'].item() for x in custom_lm.score(tmp_string)]
print(score)

[-1.4300748109817505, -1.7863236665725708, -1.6854327917099, -1.6432011127471924, -1.6571078300476074, -1.7879468202590942, -1.8273859024047852, -1.6911218166351318, -1.6490498781204224, -1.6866108179092407, -1.7833352088928223, -1.6709933280944824, -1.692547082901001, -1.7468184232711792, -1.6804587841033936, -1.8105361461639404, -1.7329291105270386, -1.7314627170562744, -1.6447081565856934, -1.7520016431808472, -1.6835275888442993, -1.6533275842666626, -1.6672742366790771, -1.6592875719070435, -1.684422254562378, -1.6835284233093262, -1.599609375, -1.5680079460144043, -1.6503944396972656, -1.753185749053955, -1.647818922996521, -1.709039330482483, -1.5755046606063843, -1.6546794176101685, -1.717308759689331, -1.6337529420852661, -1.6332213878631592, -1.6758097410202026, -1.7071884870529175, -1.6378215551376343, -1.7218457460403442, -1.6788103580474854, -1.6440554857254028, -1.7222388982772827, -1.6640251874923706, -1.6943453550338745, -1.6922346353530884, -1.7473279237747192, -1.7268

In [123]:
assert len(score) == len(tmp_string)

In [124]:
# Выбор 30 лучших кусочков
tmp = list(zip(bars, score))
tmp.sort(key=lambda x: x[1], reverse=True)
tmp = tmp[:30]
best_score = tmp[0][1]
print(best_score)

-1.4300748109817505


In [127]:
res = []

for x in tmp:
    if best_score - x[1] < 0.1:
        res.append(x[0])

print(len(res), ' melodies chosen')

1  melodies chosen


In [129]:
res

[('NOT bar_X Pos_0 Pitch_60 Dur_2 NOT bar_X Pos_2 Pitch_62 Dur_2 HALF bar_X Pos_4 Pitch_64 Dur_6 NOT bar_X Pos_10 Pitch_60 Dur_2 NOT bar_X Pos_12 Pitch_64 Dur_2 NOT bar_X Pos_14 Pitch_62 Dur_1 NOT bar_X Pos_15 Pitch_64 Dur_1 ',
  'C: C: C:')]

# Fill template: closer look

Заполнение bar_X

In [ ]:
sbar_, _ = fill_template(sbar[0], bar_idx)

In [111]:
sbar = bars[0]

In [112]:
s_bar = sbar[0]
print(s_bar)

NOT bar_X Pos_0 Pitch_60 Dur_2 NOT bar_X Pos_2 Pitch_62 Dur_2 HALF bar_X Pos_4 Pitch_64 Dur_6 NOT bar_X Pos_10 Pitch_60 Dur_2 NOT bar_X Pos_12 Pitch_64 Dur_2 NOT bar_X Pos_14 Pitch_62 Dur_1 NOT bar_X Pos_15 Pitch_64 Dur_1 


In [115]:
bar_idx

0

In [113]:
notes = s_bar.split(' ')
tmp = []
last_bar_idx = notes[1][4:]
print('Last bar index: ', last_bar_idx)

Last bar index:  X


In [116]:
# Замена X на номер такта
for i in range(len(notes)):
    if i % 5 == 1: #Каждая пятая
        if notes[i][4:] != last_bar_idx:
            bar_idx += 1
            last_bar_idx = notes[i][4:]
        tmp.append('bar_' + str(bar_idx))
    else:
        tmp.append(notes[i])

' '.join(tmp), bar_idx + 1

('NOT bar_0 Pos_0 Pitch_60 Dur_2 NOT bar_0 Pos_2 Pitch_62 Dur_2 HALF bar_0 Pos_4 Pitch_64 Dur_6 NOT bar_0 Pos_10 Pitch_60 Dur_2 NOT bar_0 Pos_12 Pitch_64 Dur_2 NOT bar_0 Pos_14 Pitch_62 Dur_1 NOT bar_0 Pos_15 Pitch_64 Dur_1 ',
 1)

# Splice: closer look

Cancatenate bar1 and bar2
In bar1, bar index is replaced while in bar2 X or Y is remained like 'X {} {} {} {} '.format(pos, pitch,dur,cadence)

In [ ]:
s_bar = splice(s_bar, last_bar)

In [203]:
bar1 = s_bar
bar2 = last_bar

print('Bar 1: ', bar1)
print('Bar 2: ', bar2)

Bar 1:  
Bar 2:  NOT bar_X Pos_4 Pitch_67 Dur_2 NOT bar_X Pos_6 Pitch_67 Dur_2 NOT bar_X Pos_8 Pitch_67 Dur_4 NOT bar_X Pos_12 Pitch_65 Dur_2 NOT bar_X Pos_14 Pitch_64 Dur_4 


In [204]:
if bar1 == '':
    s_bar = bar2
    print('bar1 is empty')
if bar2 == '':
    s_bar = bar1
    print('bar2 is empty')

bar1 is empty


если вывод есть, то на этом всё

In [137]:
assert bar1[-1] == ' '  # For the ease of concatenation, there's a space at the end of bar
assert bar2[-1] == ' '

In [138]:
notes1 = bar1.split(' ')[:-1]
notes2 = bar2.split(' ')[:-1]

In [141]:
# If the last note ending time in bar1 is not far from the begining time of the first note in bar2, just return bar1 + bar2
# Calculate the note intervals in bars. If interval between two bars <= the average interval inside a bar, 
# then it is regarded as 'not far away'.
def get_interval(notes):
    begin = []
    dur = []

    if notes[1][4:] != 'X' and notes[1][4:] != 'Y':
        start_bar = int(notes[1][4:])
    else:
        start_bar = 0

    for idx in range(len(notes) // 5):
        if notes[5 * idx + 1][4:] == 'X':
            bar_idx_ = 0
        elif notes[5 * idx + 1][4:] == 'Y':
            bar_idx_ = 1
        else:
            bar_idx_ = int(notes[5 * idx + 1][4:])
        begin.append(16 * (bar_idx_ - start_bar) + int(notes[5 * idx + 2][4:]))
        dur.append(int(notes[5 * idx + 4][4:]))
    end = list(np.array(begin) + np.array(dur))

    return list(np.array(begin[1:]) - np.array(end[:-1])), begin[0], end[-1] - 16 if end[-1] > 16 else end[-1]

In [148]:
inter1, _, end1 = get_interval(notes1)
inter2, begin2, _ = get_interval(notes2)

In [142]:
get_interval(notes1)

([0, 0, 0, 0, 0, 0], 0, 16)

In [143]:
get_interval(notes2)

([0, 0, 0, 0, 0, 0], 0, 16)

In [149]:
def avg(notes):
    return sum(notes) / len(notes)

avg_interval = avg(inter1 + inter2)
avg_interval

0.0

In [150]:
last_bar1_idx = int(notes1[-4][4:])

ValueError: invalid literal for int() with base 10: 'X'

In [ ]:
def splice(bar1, bar2):
    """
        Cancatenate bar1 and bar2
        In bar1, bar index is replaced while in bar2 X or Y is remained like 'X {} {} {} {} '.format(pos, pitch,dur,cadence)
    """
    if bar1 == '':
        return bar2
    if bar2 == '':
        return bar1

    assert bar1[-1] == ' '  # For the ease of concatenation, there's a space at the end of bar
    assert bar2[-1] == ' '
    notes1 = bar1.split(' ')[:-1]
    notes2 = bar2.split(' ')[:-1]
    bar_cnt = len(set(notes1[1::5]))

    # If the last note ending time in bar1 is not far from the begining time of the first note in bar2, just return bar1 + bar2
    # Calculate the note intervals in bars. If interval between two bars <= the average interval inside a bar, then it is regarded as 'not far away'.
    def get_interval(notes):
        begin = []
        dur = []

        if notes[1][4:] != 'X' and notes[1][4:] != 'Y':
            start_bar = int(notes[1][4:])
        else:
            start_bar = 0

        for idx in range(len(notes) // 5):
            if notes[5 * idx + 1][4:] == 'X':
                bar_idx_ = 0
            elif notes[5 * idx + 1][4:] == 'Y':
                bar_idx_ = 1
            else:
                bar_idx_ = int(notes[5 * idx + 1][4:])
            begin.append(16 * (bar_idx_ - start_bar) + int(notes[5 * idx + 2][4:]))
            dur.append(int(notes[5 * idx + 4][4:]))
        end = list(np.array(begin) + np.array(dur))

        return list(np.array(begin[1:]) - np.array(end[:-1])), begin[0], end[-1] - 16 if end[-1] > 16 else end[-1]

    inter1, _, end1 = get_interval(notes1)
    inter2, begin2, _ = get_interval(notes2)

    def avg(notes):
        return sum(notes) / len(notes)

    avg_interval = avg(inter1 + inter2)

    last_bar1_idx = int(notes1[-4][4:])
    bar2, _ = fill_template(bar2, last_bar1_idx + 1)

    if avg_interval < (16 - end1 + begin2):
        # If interval between two bars is big, shift the second bar forward.
        notes2 = bar2.split(' ')[:-1]
        tmp = ''
        for idx in range(len(notes2) // 5):
            pos = int(notes2[5 * idx + 2][4:]) - (16 - end1 + begin2)
            bar_idx_ = int(notes2[5 * idx + 1][4:])
            if pos < 0:
                bar_idx_ += pos // 16
                pos = pos % 16
            tmp += '{} bar_{} Pos_{} {} {} '.format(notes2[5 * idx], bar_idx_, pos, notes2[5 * idx + 3],
                                                    notes2[5 * idx + 4])

        return bar1 + tmp
    else:

        return bar1 + bar2

# Polish chord: closer look

Align chords and the bar. When this function is called, the bar index is already replaced by the true index instead of X or Y.
In our setting, there's 2 chords in a bar. Therefore for any position % 8==0, we write a chord.
Of course, you can modify this setting as needed.

In [ ]:
s_chord, chord_ptr = polish_chord(s_bar, s_chord, chord, chord_ptr)

In [216]:
bar = s_bar
schord = s_chord
print('Bar: ', s_bar)
print('S chord: ', schord)
print('Chords: ', chord)
print('Chord ptr: ', chord_ptr)

Bar:  NOT bar_0 Pos_6 Pitch_60 Dur_2 NOT bar_0 Pos_8 Pitch_60 Dur_2 NOT bar_0 Pos_10 Pitch_62 Dur_2 NOT bar_0 Pos_12 Pitch_64 Dur_2 NOT bar_0 Pos_14 Pitch_64 Dur_2 
S chord:   C: C: C:
Chords:  C: E:m G: B:m C: E:m G: B:m C: E:m G: B:m C: E:m G: B:m C: E:m G: B:m C: E:m G: B:m C: E:m G: B:m C: E:m G: B:m C: E:m G: B:m C: E:m G: B:m C: E:m G: B:m C: E:m G: B:m C: E:m G: B:m C:
Chord ptr:  0


In [232]:
# Урезаем аккорды
schord_list = chord_truc(bar, schord)
schord_list

['', 'C:']

In [233]:
last_chord = schord_list[-1]
schord = ' '.join(schord_list)
print(schord)

 C:


In [234]:
chord_list = chord.split(' ')

In [235]:
if last_chord not in chord_list:
    chord_ptr = (chord_ptr + 1) % len(chord_list)
    print(chord_ptr)

Если ничего не вывелось, то:

In [236]:
chord_ptr = (chord_list.index(last_chord) + 1) % len(chord_list)
print(chord_ptr)

1


return schord, chord_ptr

# Chord truc: closer look

Given a bar string, remove redundant chords.

In [ ]:
schord_list = chord_truc(bar, schord)

In [226]:
schord_list = schord.split(' ')
schord_list

['', 'C:', 'C:', 'C:']

In [227]:
notes = bar.strip().split(' ')
print(*notes[:10])

NOT bar_0 Pos_6 Pitch_60 Dur_2 NOT bar_0 Pos_8 Pitch_60 Dur_2


In [228]:
start_pos = 16 * int(notes[1][4:]) + int(notes[2][4:])
end_pos = 16 * int(notes[-4][4:]) + int(notes[-3][4:]) + int(notes[-1][4:])
duration = end_pos - start_pos
chord_num = ceil(duration / 8)
print('From ', start_pos, ' to ', end_pos)
print('Duration: ', duration)
print('Chord number: ', chord_num)

From  6  to  16
Duration:  10
Chord number:  2


In [229]:
assert chord_num >= 1, 'bar:{},chord:{}'.format(bar, schord)

In [230]:
if len(schord_list) >= chord_num:
    schord_list = schord_list[:chord_num]
    print('Now: ', schord_list)

Now:  ['', 'C:']


Если ничего не вывелось, то:

In [ ]:
tmp = []
for i in schord_list:
        tmp.append(i)
        tmp.append(i)
    schord_list = tmp[:chord_num]

Выводится schord list

# Polish: closer look

Three fuctions:
1. Avoid bars overlapping.
2. Make the first note in all bars start at the position 0.
3. Remove rest and cadence in a bar.

In [ ]:
s_bar, bar_idx, last_note_end = polish(s_bar, last_note_end)

In [265]:
bar = s_bar
print(bar)

NOT bar_0 Pos_6 Pitch_60 Dur_2 NOT bar_0 Pos_8 Pitch_60 Dur_2 NOT bar_0 Pos_10 Pitch_62 Dur_2 NOT bar_0 Pos_12 Pitch_64 Dur_2 NOT bar_0 Pos_14 Pitch_64 Dur_2 NOT bar_1 Pos_0 Pitch_71 Dur_2 NOT bar_1 Pos_2 Pitch_67 Dur_2 NOT bar_1 Pos_4 Pitch_64 Dur_4 NOT bar_1 Pos_8 Pitch_71 Dur_2 NOT bar_1 Pos_10 Pitch_67 Dur_2 NOT bar_1 Pos_12 Pitch_64 Dur_4 NOT bar_2 Pos_0 Pitch_62 Dur_2 HALF bar_2 Pos_2 Pitch_59 Dur_6 NOT bar_2 Pos_14 Pitch_55 Dur_2 


In [266]:
last_note_end

-16

In [267]:
notes = bar.strip().split(' ')
tmp = ''
is_tuned = False

In [268]:
for idx in range(len(notes) // 5):
    pos = int(notes[5 * idx + 2][4:])
    bar_idx_ = int(notes[5 * idx + 1][4:])
    dur = int(notes[5 * idx + 4][4:])

    print('pos: ', pos)
    print('bar index: ', bar_idx_)
    print('duration: ', dur)

    this_note_start = 16 * bar_idx_ + pos

    print('this not start', this_note_start)

    cadence = 'NOT'
    if idx == 0:
        blank_after_last_note = 16 - last_note_end % 16
        threshold = blank_after_last_note
        print('threshold: ', threshold)
    else:
        threshold = 0
        print('threshold: ', threshold)

    # максимальная длительность - 1\2, минимальная - 1\8
    if dur == 1:  # the minimum granularity is a 1/8 note.
        dur = 2
        print('Changed duration to ', dur)
    if dur > 8:  # the maximum granularity is a 1/2 note.
        dur = 8
        print('Changed duration to ', dur)

     # Function 3: removing ret and cadence in a bar
    if this_note_start - last_note_end != threshold:
        pos += (last_note_end + threshold - this_note_start)
        bar_idx_ += pos // 16
        pos = pos % 16
        print('changed pos to ', pos)
        print('changed bar index to ', bar_idx_)


    if idx == len(notes) // 5 - 2:
        if 12 < pos + dur <= 16 or len(notes) // 5 <= 4:
            dur = 16 - pos
            is_tuned = True
            print('changed duration to ', dur)


    if idx == len(notes) // 5 - 1:
        if is_tuned:
            print('is tuned')
            pos = 0
            print('changed pos to ', pos)
        else:
            if 12 < pos + dur <= 16:
                dur += 6
                print('changed duration to ', dur)
        cadence = 'HALF'  # just for the ease of model scoring
        print('making a half cadence')

    last_note_end = 16 * bar_idx_ + pos + dur  
    print('last note end: ', last_note_end)

    assert pos <= 16
    tmp += '{} bar_{} Pos_{} {} Dur_{} '.format(cadence, bar_idx_, pos, notes[5 * idx + 3], dur)

pos:  6
bar index:  0
duration:  2
this not start 6
threshold:  16
changed pos to  0
changed bar index to  0
last note end:  2
pos:  8
bar index:  0
duration:  2
this not start 8
threshold:  0
changed pos to  2
changed bar index to  0
last note end:  4
pos:  10
bar index:  0
duration:  2
this not start 10
threshold:  0
changed pos to  4
changed bar index to  0
last note end:  6
pos:  12
bar index:  0
duration:  2
this not start 12
threshold:  0
changed pos to  6
changed bar index to  0
last note end:  8
pos:  14
bar index:  0
duration:  2
this not start 14
threshold:  0
changed pos to  8
changed bar index to  0
last note end:  10
pos:  0
bar index:  1
duration:  2
this not start 16
threshold:  0
changed pos to  10
changed bar index to  0
last note end:  12
pos:  2
bar index:  1
duration:  2
this not start 18
threshold:  0
changed pos to  12
changed bar index to  0
last note end:  14
pos:  4
bar index:  1
duration:  4
this not start 20
threshold:  0
changed pos to  14
changed bar index 

In [269]:
print(tmp)
bar_idx_ + 1

NOT bar_0 Pos_0 Pitch_60 Dur_2 NOT bar_0 Pos_2 Pitch_60 Dur_2 NOT bar_0 Pos_4 Pitch_62 Dur_2 NOT bar_0 Pos_6 Pitch_64 Dur_2 NOT bar_0 Pos_8 Pitch_64 Dur_2 NOT bar_0 Pos_10 Pitch_71 Dur_2 NOT bar_0 Pos_12 Pitch_67 Dur_2 NOT bar_0 Pos_14 Pitch_64 Dur_4 NOT bar_1 Pos_2 Pitch_71 Dur_2 NOT bar_1 Pos_4 Pitch_67 Dur_2 NOT bar_1 Pos_6 Pitch_64 Dur_4 NOT bar_1 Pos_10 Pitch_62 Dur_2 NOT bar_1 Pos_12 Pitch_59 Dur_6 HALF bar_2 Pos_2 Pitch_55 Dur_2 


3

In [270]:
last_note_end

36

# Not mono: closer look

Filter monotonous pieces.

In [ ]:
if not_mono(s_bar) and not_duplicate(s_bar, select_notes[-1]):
    s_chord = last_chord + ' ' + s_chord
    break

In [297]:
bar = s_bar
notes = bar.split(' ')[:-1][3::5]
notes = [int(x[6:]) for x in notes]
print(notes)

[60, 60, 62, 64, 64, 71, 67, 64, 71, 71, 71, 76, 78, 74]


In [298]:
# Заполняем единицами слоты, соответствующие использованным в мелодии высотам
tmp = [0] * 128
for idx in range(len(notes)):
    tmp[int(notes[idx])] = 1

In [300]:
# Если мелодия состоит меньше, чем из четырёх нот и они все одинаковые по высоте
# Или если мелодия длинная, но используется в ней меньше трйх высот
# То мелодия монотонная
if (1 < len(notes) <= 3 and sum(tmp) == 1) or (len(notes) >= 4 and sum(tmp) < 3):
    print('The melody is monotonous')
else:
    print('The melody is not monotonous')

The melody is not monotonous


Не факт, что это разумный подход, так как монотонность может звучать ярко при параллельной смене гармоний

# not duplicate: closer look

De-duplication, only care about the pitch.

In [301]:
bar1 = s_bar
bar2 = select_notes[-1]

In [302]:
notes1 = bar1.split(' ')[:-1][3::5]
notes2 = bar2.split(' ')[:-1][3::5]
print('notes 1: ', notes1)
print('notes 2: ', notes2)

notes 1:  ['Pitch_60', 'Pitch_60', 'Pitch_62', 'Pitch_64', 'Pitch_64', 'Pitch_71', 'Pitch_67', 'Pitch_64', 'Pitch_71', 'Pitch_71', 'Pitch_71', 'Pitch_76', 'Pitch_78', 'Pitch_74']
notes 2:  ['Pitch_60', 'Pitch_60', 'Pitch_62', 'Pitch_64', 'Pitch_64', 'Pitch_71', 'Pitch_67', 'Pitch_64', 'Pitch_71', 'Pitch_67', 'Pitch_64', 'Pitch_62', 'Pitch_59', 'Pitch_55']


In [303]:
if notes1 != notes2:
    print('Melodies are not duplicates')
else:
    print('Melodies are duplicates')

Melodies are not duplicates


# save demo: closer look

In [ ]:
save_demo(
    note_string, 
    select_chords, 
    name, 
    lang= 'en',
    lyrics.strip().split('[sep]'),
    [len(i) for i in sentence])

In [11]:
notes_str = note_string
sentence = lyrics.strip().split('[sep]')
word_counter = [len(i) for i in sentence]

In [12]:
pitch_dict = {'C': 0, 'C#': 1, 'D': 2, 'Eb': 3, 'E': 4, 'F': 5, 'F#': 6, 'G': 7, 'Ab': 8, 'A': 9, 'Bb': 10, 'B': 11}
_CHORD_KIND_PITCHES = {
        '': [0, 4, 7],
        'm': [0, 3, 7],
        '+': [0, 4, 8],
        'dim': [0, 3, 6],
        '7': [0, 4, 7, 10],
        'maj7': [0, 4, 7, 11],
        'm7': [0, 3, 7, 10],
        'm7b5': [0, 3, 6, 10],
    }

In [13]:
name = 'test_2'
name

'test_2'

In [14]:
mf = MIDIFile(2)  # only 1 track
melody_track = 0  # the only track
chord_track = 1
time = 0  # start at the beginning
channel = 0

In [15]:
mf.addTrackName(melody_track, time, "melody")
mf.addTrackName(chord_track, time, "chord")
mf.addTempo(melody_track, time, 120) #can be modificated
mf.addTempo(chord_track, time, 120)

In [16]:
notes = notes_str.split(' ')
cnt = 0
sen_idx = 0
chord_time = []

In [17]:
for i in range(len(notes) // 5):
    print('writing idx: ', i)
    # cadence = notes[5 * i]
    bar = int(notes[5 * i + 1][4:])
    pos = int(notes[5 * i + 2][4:])  # // pos_resolution
    pitch = int(notes[5 * i + 3][6:])
    dur = int(notes[5 * i + 4][4:]) / 4

    time = bar * 4 + pos / 4  # + delta

    # if cadence == 'HALF':
    #     delta += 2
    # if cadence == 'AUT':
    #    delta += 4

    mf.addNote(melody_track, channel, pitch, time, dur, 100)

    # fill all chords into bars before writing notes
    if cnt == 0:
        cds = select_chords[sen_idx].split(' ')
        t = time - time % 2
        if len(chord_time) > 0:
            blank_dur = t - chord_time[-1] - 2
            insert_num = int(blank_dur / 2)

            print('Chords:', cds[0].split(':'))
            root, cd_type = cds[0].split(':')
            root = pitch_dict[root]
            for i in range(insert_num):
                for shift in _CHORD_KIND_PITCHES[cd_type]:
                    mf.addNote(chord_track, channel, 36 + root + shift, chord_time[-1] + 2, 2, 75)
                chord_time.append(chord_time[-1] + 2)

        print('begin sentence:', sen_idx)
        for cd in cds:
            root, cd_type = cd.split(':')
            root = pitch_dict[root]
            # mf.addNote(chord_track, channel, 36+root, t, 2, 75)  # 36 is C3
            for shift in _CHORD_KIND_PITCHES[cd_type]:
                mf.addNote(chord_track, channel, 36 + root + shift, t, 2, 75)
            chord_time.append(t)
            t += 2

    cnt += 1
    if cnt == word_counter[sen_idx]:
        cnt = 0
        sen_idx += 1

writing idx:  0
begin sentence: 0
writing idx:  1
writing idx:  2
writing idx:  3
writing idx:  4
writing idx:  5
writing idx:  6
writing idx:  7
writing idx:  8
writing idx:  9
writing idx:  10
writing idx:  11
writing idx:  12
writing idx:  13
writing idx:  14
writing idx:  15
writing idx:  16
writing idx:  17
writing idx:  18
writing idx:  19
writing idx:  20
writing idx:  21
writing idx:  22
writing idx:  23
writing idx:  24
writing idx:  25
writing idx:  26
writing idx:  27
writing idx:  28
writing idx:  29
writing idx:  30
writing idx:  31
writing idx:  32
writing idx:  33
writing idx:  34
writing idx:  35
writing idx:  36
writing idx:  37
writing idx:  38
writing idx:  39
writing idx:  40
Chords: ['G', '']
begin sentence: 1
writing idx:  41
writing idx:  42
writing idx:  43
writing idx:  44
writing idx:  45
writing idx:  46
writing idx:  47
writing idx:  48
writing idx:  49
writing idx:  50
writing idx:  51
writing idx:  52
writing idx:  53
writing idx:  54
writing idx:  55
writ

In [18]:
name += '.mid'
with open(name, 'wb') as outf:
    mf.writeFile(outf)

Теперь добавим текст в мидифайл

In [20]:
name = 'test_2.mid'
midi_obj = miditoolkit.midi.parser.MidiFile(name)

In [21]:
print(sentence[0])
lyrics = ' '.join(sentence).split(' ')
print(lyrics[:15])

Кто зна ет, за чем за жи га ют звёз ды? 
['Кто', 'зна', 'ет,', 'за', 'чем', 'за', 'жи', 'га', 'ют', 'звёз', 'ды?', '', '', 'Для', 'ко']


In [22]:
word_idx = 0

In [23]:
for word in lyrics:
    if word not in [',', '.', '']:
        note = midi_obj.instruments[0].notes[word_idx]
        midi_obj.lyrics.append(
            miditoolkit.Lyric(text=word, time=note.start))
        word_idx += 1
    else:
        midi_obj.lyrics[-1].text += word    

In [24]:
print(midi_obj.lyrics[:5])

[Lyric(text="Кто", time=0), Lyric(text="зна", time=480), Lyric(text="ет,", time=960), Lyric(text="за", time=1680), Lyric(text="чем", time=2160)]


In [25]:
midi_obj.dump(f'{name}', charset='utf-8')

In [32]:
midi_obj.__dict__

{'ticks_per_beat': 960,
 'tempo_changes': [TempoChange(tempo=120.0, time=0)],
 'key_signature_changes': [],
 'time_signature_changes': [],
 'markers': [],
 'lyrics': [Lyric(text="Take", time=0),
  Lyric(text="a", time=480),
  Lyric(text="look", time=960),
  Lyric(text="in", time=1440),
  Lyric(text="side", time=2400),
  Lyric(text="your", time=2880),
  Lyric(text="heart,", time=3360),
  Lyric(text="is", time=3840),
  Lyric(text="there", time=4320),
  Lyric(text="a", time=4800),
  Lyric(text="ny", time=5520),
  Lyric(text="room", time=6000),
  Lyric(text="for", time=6480),
  Lyric(text="me?", time=7680),
  Lyric(text="I", time=11520),
  Lyric(text="won_t", time=12000),
  Lyric(text="have", time=12480),
  Lyric(text="to", time=12960),
  Lyric(text="hold", time=13920),
  Lyric(text="my", time=14400),
  Lyric(text="breath", time=14880),
  Lyric(text="till", time=15360),
  Lyric(text="you", time=15840),
  Lyric(text="get", time=16320),
  Lyric(text="down", time=17040),
  Lyric(text="on", ti

# Reading MIDI files

In [2]:
import abjad
note = abjad.Note("c'4")
abjad.show(note)

/home/marykorol/bin/lilypond: 4: exec: /home/marykorol/lilypond/usr/bin/lilypond: Permission denied


IndexError: list index out of range

In [3]:
import abjad
string = "c'16 f' g' a' d' g' a' b' e' a' b' c'' f' b' c'' d''16"
voice_1 = abjad.Voice(string, name="Voice_1")
staff_1 = abjad.Staff([voice_1], name="Staff_1")
abjad.show(staff_1)

/home/marykorol/bin/lilypond: 4: exec: /home/marykorol/lilypond/usr/bin/lilypond: Permission denied



In [4]:
string = "d'8 f' a' d'' f'' gs'4 r8 e' gs' b' e'' gs'' a'4"
voice = abjad.Voice(string, name="RH_Voice")
staff = abjad.Staff([voice], name="RH_Staff")
score = abjad.Score([staff], name="Score")
abjad.show(score)

/home/marykorol/bin/lilypond: 4: exec: /home/marykorol/lilypond/usr/bin/lilypond: Permission denied

